In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Configuración de gráficos
plt.style.use('ggplot')
pd.set_option('display.max_columns', None)

# --- 1. CARGA DE DATOS ---
file_path = 'datos_recolector.csv' 
output_path = 'datos_limpios.csv'

try:
    df = pd.read_csv(file_path)
    print(f"1. Datos cargados: {df.shape[0]} filas, {df.shape[1]} columnas")
except FileNotFoundError:
    print("Error: No se encuentra el archivo.")
    raise

cols_sensores = [c for c in df.columns if c.startswith('S_')]
cols_acciones = [c for c in df.columns if c.startswith('A_')]

X = df[cols_sensores]
y = df[cols_acciones]

# --- 2. LIMPIEZA AUTOMÁTICA (Columnas Constantes) ---
std_dev = X.std()
cols_constantes = std_dev[std_dev == 0].index.tolist()
if cols_constantes:
    print(f"\n2. Eliminando {len(cols_constantes)} columnas de sensores constantes.")
    X = X.drop(columns=cols_constantes)

# --- 2.5 LIMPIEZA MANUAL (Tu petición) ---
sensores_basura = ['S_damage', 'S_distFromStart', 'S_lastLapTime', 'S_distRaced', 'S_fuel']
a_borrar_manual = [c for c in sensores_basura if c in X.columns]

if a_borrar_manual:
    print(f"\n2.5 Eliminando sensores manuales: {a_borrar_manual}")
    X = X.drop(columns=a_borrar_manual)

# --- 3. LIMPIEZA AUTOMÁTICA (Columnas Redundantes) ---
UMBRAL_CORRELACION = 0.95 
print(f"\n3. Analizando correlaciones entre columnas (Umbral: {UMBRAL_CORRELACION*100}%)...")

corr_matrix = X.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > UMBRAL_CORRELACION)]

if to_drop:
    print(f"   Eliminando {len(to_drop)} sensores redundantes.")
    X = X.drop(columns=to_drop)
else:
    print("   No se encontraron sensores redundantes.")

# --- 4. UNIFICACIÓN Y LIMPIEZA DE FILAS (NUEVO) ---
print("-" * 60)
print("4. Procesando filas...")

# Unimos todo en el dataframe final
df_final = pd.concat([X, y], axis=1)

# Contamos cuántas hay antes de limpiar
filas_antes = df_final.shape[0]

# --- AQUÍ ELIMINAMOS LAS FILAS DUPLICADAS ---
df_final = df_final.drop_duplicates()

filas_despues = df_final.shape[0]
filas_borradas = filas_antes - filas_despues

if filas_borradas > 0:
    print(f"   [!] Se eliminaron {filas_borradas} FILAS duplicadas.")
else:
    print("   No se encontraron filas duplicadas.")

# --- 5. GUARDAR ---
df_final.to_csv(output_path, index=False)

print("-" * 60)
print(f"¡ÉXITO! Archivo '{output_path}' generado.")
print(f"Dimensiones finales: {df_final.shape[0]} filas x {df_final.shape[1]} columnas")
print("Columnas finales de sensores:")
print(list(X.columns))

1. Datos cargados: 125636 filas, 38 columnas

2. Eliminando 5 columnas de sensores constantes.

3. Analizando correlaciones entre columnas (Umbral: 95.0%)...
   Eliminando 6 sensores redundantes.
------------------------------------------------------------
4. Procesando filas...
   [!] Se eliminaron 104 FILAS duplicadas.
------------------------------------------------------------
¡ÉXITO! Archivo 'datos_limpios.csv' generado.
Dimensiones finales: 125532 filas x 27 columnas
Columnas finales de sensores:
['S_angle', 'S_gear', 'S_rpm', 'S_speed', 'S_track_0', 'S_track_3', 'S_track_4', 'S_track_5', 'S_track_6', 'S_track_7', 'S_track_8', 'S_track_9', 'S_track_10', 'S_track_11', 'S_track_12', 'S_track_13', 'S_track_14', 'S_track_15', 'S_track_16', 'S_track_17', 'S_trackPos', 'S_wheelSpinVel_0', 'S_z']


In [2]:
# ==============================================================================
# AUDITORÍA DEFINITIVA: EL "TRIPLE JUEZ" (CORRELACIÓN + MI + RANDOM FOREST)
# ==============================================================================

# Definición de colores ANSI
C_RED = '\033[91m'
C_YELLOW = '\033[93m'
C_GREEN = '\033[92m'
C_RESET = '\033[0m'

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_selection import mutual_info_regression
from sklearn.ensemble import RandomForestRegressor
import numpy as np

# 0. SINCRONIZACIÓN DE SEGURIDAD (Para evitar el error de longitud)
# ------------------------------------------------------------------------------
if isinstance(X, np.ndarray):
    X = pd.DataFrame(X, columns=[f'S_{i}' for i in range(X.shape[1])])
if isinstance(y, np.ndarray):
    nombres_y = ['A_Accel',"A_Brake", 'A_Gear', 'A_Steer']
    if y.shape[1] == 4:
        y = pd.DataFrame(y, columns=nombres_y)
    else:
        y = pd.DataFrame(y, columns=[f'Action_{i}' for i in range(y.shape[1])])

common_indices = X.index.intersection(y.index)
X = X.loc[common_indices]
y = y.loc[common_indices]
print(f"✅ Datos sincronizados: {len(X)} filas analizadas.")

# ------------------------------------------------------------------------------
# BUCLE DE ANÁLISIS POR ACCIÓN
# ------------------------------------------------------------------------------
acciones = y.columns.tolist() 

for accion in acciones:
    print(f"\n" + "="*60)
    print(f" 🔍 ANALIZANDO: {accion.upper()}")
    print("="*60)
    
    # --- JUEZ 1: CORRELACIÓN LINEAL (PEARSON) ---
    print("   1. Calculando Correlación Lineal...")
    # Calculamos solo la columna que nos interesa para ahorrar memoria
    # Usamos .abs() porque nos importa la fuerza, no si es positiva o negativa
    corr_scores = X.apply(lambda col: col.corr(y[accion])).abs().fillna(0)
    
    # --- JUEZ 2: MUTUAL INFORMATION (NO LINEAL) ---
    print("   2. Calculando Información Mutua (Complejidad)...")
    mi_raw = mutual_info_regression(X, y[accion], random_state=42)
    mi_scores = pd.Series(mi_raw, index=X.columns)
    
    # --- JUEZ 3: RANDOM FOREST (IMPORTANCIA REAL) ---
    print("   3. Entrenando Random Forest (Prueba de fuego)...")
    rf = RandomForestRegressor(n_estimators=50, random_state=42, n_jobs=-1)
    rf.fit(X, y[accion])
    rf_scores = pd.Series(rf.feature_importances_, index=X.columns)
    
    # --------------------------------------------------------------------------
    # CÁLCULO DE LA NOTA FINAL (PROMEDIO NORMALIZADO)
    # --------------------------------------------------------------------------
    # Normalizamos todo de 0 a 1 para poder sumarlo
    def normalizar(series):
        return (series - series.min()) / (series.max() - series.min())

    score_corr = normalizar(corr_scores)
    score_mi   = normalizar(mi_scores)
    score_rf   = normalizar(rf_scores)
    
    # PROMEDIO: Damos el mismo peso a los 3 jueces (33% cada uno)
    nota_final = (score_corr + score_mi + score_rf) / 3
    
    # Ordenamos de mejor a peor
    ranking = nota_final.sort_values(ascending=False).head(30)
    
    # --------------------------------------------------------------------------
    # VISUALIZACIÓN
    # --------------------------------------------------------------------------
    print(f"\n🏆 TOP 30 SENSORES PARA {accion} (Ranking Combinado):")
    print("-" * 50)
    print(f"{'SENSOR':<15} | {'NOTA FINAL':<10} | {'Corr':<6} | {'MI':<6} | {'RF':<6}")
    print("-" * 50)
    
    for sensor, nota in ranking.items():
        # Recuperamos los valores originales normalizados para ver quién votó qué
        c = score_corr[sensor]
        m = score_mi[sensor]
        r = score_rf[sensor]
        # LOGICA DE COLORES
       # Función auxiliar para decidir el color de un valor concreto
        def obtener_color(valor):
            if valor >= 0.8:
                return C_GREEN    # >= 0.8: Verde
            elif valor >= 0.5:
                return C_RESET    # 0.5 a 0.79: Blanco (Normal)
            elif valor >= 0.3:
                return C_YELLOW   # 0.3 a 0.49: Amarillo
            else:
                return C_RED      # < 0.3: Rojo

        # Calculamos el color INDEPENDIENTE para cada columna
        col_n = obtener_color(nota)
        col_c = obtener_color(c)
        col_m = obtener_color(m)
        col_r = obtener_color(r)
            
        # Imprimimos: El nombre del sensor se queda en blanco, los números van con su color
        # Nota: Ponemos {C_RESET} después de cada número para que no pinte al siguiente
        print(f"{sensor:<15} | {col_n}{nota:.4f}{C_RESET}     | {col_c}{c:.2f}{C_RESET}   | {col_m}{m:.2f}{C_RESET}   | {col_r}{r:.2f}{C_RESET}")    
   
    # Generamos la lista lista para copiar
    print(f"\n📋 LISTA PARA COPIAR ({accion}):")
    lista_str = ", ".join([f"feature_idx['{s}']" for s in ranking.index])
    print(f"[{lista_str}]")

✅ Datos sincronizados: 125636 filas analizadas.

 🔍 ANALIZANDO: A_ACCEL
   1. Calculando Correlación Lineal...
   2. Calculando Información Mutua (Complejidad)...
   3. Entrenando Random Forest (Prueba de fuego)...

🏆 TOP 30 SENSORES PARA A_Accel (Ranking Combinado):
--------------------------------------------------
SENSOR          | NOTA FINAL | Corr   | MI     | RF    
--------------------------------------------------
S_track_9       | 0.8532     | 1.00   | 0.56   | 1.00
S_speed         | 0.7279     | 0.44   | 1.00   | 0.74
S_wheelSpinVel_0 | 0.5136     | 0.66   | 0.74   | 0.14
S_rpm           | 0.4486     | 0.78   | 0.54   | 0.02
S_track_8       | 0.4036     | 0.72   | 0.37   | 0.11
S_track_10      | 0.3956     | 0.75   | 0.34   | 0.10
S_track_4       | 0.2852     | 0.46   | 0.40   | 0.00
S_track_5       | 0.2405     | 0.43   | 0.28   | 0.01
S_track_14      | 0.2395     | 0.35   | 0.36   | 0.01
S_track_3       | 0.2392     | 0.37   | 0.34   | 0.01
S_track_11      | 0.2240     | 0.

In [1]:
import pandas as pd
import numpy as np

# Cargar datos
try:
    df = pd.read_csv('datos_limpios.csv')
    
    # Filtrar solo columnas de sensores (S_)
    cols_sensores = [c for c in df.columns if c.startswith('S_')]
    
    # Asegurar el orden correcto (segun csv header/training)
    # El orden en el CSV ya es el correcto normalmente, pero lo verificamos
    # X se crea como df[cols_sensores] en el notebook, asi que mantiene orden del CSV
    
    X = df[cols_sensores]
    
    # Calcular Mean y Std (Scale)
    scaler_mean = X.mean(axis=0).values
    scaler_scale = X.std(axis=0).values
    
    print("SCALER MEAN:")
    print(", ".join([f"{x:.8f}" for x in scaler_mean]))
    
    print("\nSCALER SCALE:")
    print(", ".join([f"{x:.8f}" for x in scaler_scale]))
    
    print(f"\nTotal columnas: {len(cols_sensores)}")
    print(f"Columnas: {cols_sensores}")

except Exception as e:
    print(f"Error: {e}")


SCALER MEAN:
-0.00205591, 2.64444126, 4415.52944382, 77.95180448, 7.61571627, 11.87461907, 19.39752355, 34.85956336, 41.98884369, 48.98707374, 54.87639929, 58.56179033, 48.84318519, 40.69321211, 33.78569296, 25.74149638, 17.39452623, 10.55262829, 7.62845053, 6.56981317, -0.04575499, 62.75078968, 0.34168918

SCALER SCALE:
0.06410456, 1.08946127, 1061.94762450, 36.68689551, 5.30684393, 10.65443270, 19.36858740, 43.89944727, 45.45835321, 44.62270662, 44.74142880, 49.63728173, 40.30549319, 35.36238996, 30.97158208, 23.30660485, 18.17308043, 10.47570956, 5.30996271, 2.97112274, 0.32603312, 32.22361920, 0.00356310

Total columnas: 23
Columnas: ['S_angle', 'S_gear', 'S_rpm', 'S_speed', 'S_track_0', 'S_track_3', 'S_track_4', 'S_track_5', 'S_track_6', 'S_track_7', 'S_track_8', 'S_track_9', 'S_track_10', 'S_track_11', 'S_track_12', 'S_track_13', 'S_track_14', 'S_track_15', 'S_track_16', 'S_track_17', 'S_trackPos', 'S_wheelSpinVel_0', 'S_z']


In [5]:
import pandas as pd
import numpy as np

# Cargar datos
try:
    print("Cargando datos...")
    df = pd.read_csv('datos_limpios.csv')
    
    # Columnas objetivo
    target_cols = ['S_speed', 'S_track_9', 'A_Accel', 'A_Brake']
    
    # Filtrar solo las columnas que realmente existen en el CSV
    cols_existentes = [c for c in target_cols if c in df.columns]
    
    if not cols_existentes:
        print(f"ERROR: No se encontraron las columnas. Disponibles: {list(df.columns)}")
    else:
        print(f"\n--- ANÁLISIS DE ESTADÍSTICAS (FILTRANDO CEROS EN ACCIONES) ---")
        
        for col in cols_existentes:
            # 1. Seleccionar la columna
            series_data = df[col]
            
            # 2. LOGICA DE FILTRADO:
            # Si es Acelerar o Frenar, eliminamos los valores que sean 0 (o muy cercanos a 0)
            if col in ['A_Accel', 'A_Brake']:
                count_total = len(series_data)
                # Filtramos valores mayores a 0.01 para evitar ruido flotante
                series_data = series_data[series_data > 0.01] 
                count_filtered = len(series_data)
                print(f"\n[Filtro activo para {col}]: Se ignoraron {count_total - count_filtered} registros donde el valor era 0.")
            
            # 3. Calcular estadísticas si quedan datos
            if series_data.empty:
                print(f"  -> La columna {col} no tiene valores mayores a 0.")
                continue

            # Usamos percentiles 33% y 66% para tener 3 estados equilibrados (Bajo, Medio, Alto)
            # O 25%, 50%, 75% si prefieres cuartiles clásicos. Dejo los clásicos:
            stats = series_data.describe(percentiles=[0.25, 0.5, 0.75])
            
            # 4. Imprimir resultados para Java
            q1 = stats['25%']
            q2 = stats['50%'] # Mediana
            q3 = stats['75%']
            
            print(f"Estadísticas para: {col}")
            print(f"  - Min: {stats['min']:.4f} | Max: {stats['max']:.4f}")
            print(f"  - Q1 (25% - Bajo) : {q1:.4f}")
            print(f"  - Q2 (50% - Media): {q2:.4f}")
            print(f"  - Q3 (75% - Alto) : {q3:.4f}")
            
            print("-" * 40)

except Exception as e:
    print(f"Error crítico: {e}")

Cargando datos...

--- ANÁLISIS DE ESTADÍSTICAS (FILTRANDO CEROS EN ACCIONES) ---
Estadísticas para: S_speed
  - Min: 0.0000 | Max: 149.6590
  - Q1 (25% - Bajo) : 47.1202
  - Q2 (50% - Media): 78.6718
  - Q3 (75% - Alto) : 101.6393
----------------------------------------
Estadísticas para: S_track_9
  - Min: 9.4448 | Max: 200.0000
  - Q1 (25% - Bajo) : 26.4871
  - Q2 (50% - Media): 42.6308
  - Q3 (75% - Alto) : 65.4784
----------------------------------------

[Filtro activo para A_Accel]: Se ignoraron 21002 registros donde el valor era 0.
Estadísticas para: A_Accel
  - Min: 0.0100 | Max: 1.0000
  - Q1 (25% - Bajo) : 0.2557
  - Q2 (50% - Media): 0.5434
  - Q3 (75% - Alto) : 1.0000
----------------------------------------

[Filtro activo para A_Brake]: Se ignoraron 110913 registros donde el valor era 0.
Estadísticas para: A_Brake
  - Min: 0.0100 | Max: 1.0000
  - Q1 (25% - Bajo) : 0.1699
  - Q2 (50% - Media): 0.4297
  - Q3 (75% - Alto) : 0.8723
----------------------------------------


In [6]:
import pandas as pd
import numpy as np
from scipy.stats import skew

def analyze_reward_function(df, col_name, target_range=(-1, 1)):
    """
    Analiza una columna y genera los parámetros para Linear y Sigmoid.
    target_range: El rango de salida deseado para la recompensa (ej: -1 a 1)
    """
    data = df[col_name].dropna()
    
    # 1. Estadísticas Básicas
    mu = data.median()      # Q2
    sigma = data.std()
    q1 = data.quantile(0.25)
    q3 = data.quantile(0.75)
    iqr = q3 - q1
    data_skew = skew(data)  # Asimetría
    
    print(f"\n>>> ANÁLISIS PARA: {col_name}")
    print(f"    Mediana (Q2): {mu:.4f}")
    print(f"    IQR (Q3-Q1):  {iqr:.4f}")
    print(f"    Asimetría:    {data_skew:.4f} (0=Normal, >1=Sesgada)")

    # 2. CALCULO DE PARÁMETROS

    # --- A) OPCIÓN LINEAL (y = mx + c) ---
    # Queremos mapear [Q1, Q3] al rango target (aprox) para ser robustos
    # min_out = m*Q1 + c
    # max_out = m*Q3 + c
    t_min, t_max = target_range
    if iqr == 0: iqr = 1e-5 # Evitar div/0
    
    m = (t_max - t_min) / iqr
    c = t_min - (m * q1)
    
    # --- B) OPCIÓN SIGMOIDE (Logística) ---
    # y = (t_max - t_min) / (1 + e^(-k*(x - center))) + t_min
    # Center es la Mediana. k define la pendiente.
    # Ajustamos k para que en Q3 la curva esté al 80-90% del recorrido
    k = 4.0 / iqr 
    center = mu

    # 3. RECOMENDACIÓN
    # Si la asimetría es alta (>1 o <-1), Linear no sirve porque castiga demasiado los extremos.
    best_option = "SIGMOIDE" if abs(data_skew) > 0.5 else "LINEAL"
    
    print(f"    RECOMENDACIÓN: La distribución es {best_option}.")
    
    return {
        "col": col_name,
        "type": best_option,
        "linear_m": m,
        "linear_c": c,
        "sig_k": k,
        "sig_center": center,
        "min_out": t_min,
        "max_out": t_max
    }

# --- MAIN ---
try:
    print("Leyendo dataset...")
    df = pd.read_csv('datos_limpios.csv')
    
    # Filtro básico
    df = df[df['S_speed'] > 1.0] # Ignorar coche parado
    
    # Analizamos Velocidad (Queremos maximizarla -> Rango 0 a 1)
    # Rango 0 a 1 porque es positivo ir rápido
    res_speed = analyze_reward_function(df, 'S_speed', target_range=(0.0, 1.0))
    
    # Analizamos Distancia (Queremos seguridad -> Rango -1 a 1)
    # Rango -1 (muro cerca) a 1 (lejos)
    res_dist = analyze_reward_function(df, 'S_track_9', target_range=(-1.0, 1.0))

    print("\n" + "="*60)
    print(" GENERADOR DE CÓDIGO JAVA (Copiar y Pegar en EnvAccel.java)")
    print("="*60)
    
    print(f"\n// Variables calculadas automáticamente para {res_speed['type']} y {res_dist['type']}")
    
    # GENERAR CÓDIGO JAVA PARA VELOCIDAD
    print(f"// --- Lógica para Velocidad ({res_speed['type']}) ---")
    if res_speed['type'] == 'LINEAL':
        print(f"double m_speed = {res_speed['linear_m']:.6f};")
        print(f"double c_speed = {res_speed['linear_c']:.6f};")
        print("// Formula: reward = m * speed + c")
    else: # Sigmoide
        print(f"double k_speed = {res_speed['sig_k']:.6f};")
        print(f"double mu_speed = {res_speed['sig_center']:.6f};")
        print(f"// Formula: Sigmoide ajustada entre {res_speed['min_out']} y {res_speed['max_out']}")

    # GENERAR CÓDIGO JAVA PARA DISTANCIA
    print(f"\n// --- Lógica para Distancia ({res_dist['type']}) ---")
    if res_dist['type'] == 'LINEAL':
        print(f"double m_dist = {res_dist['linear_m']:.6f};")
        print(f"double c_dist = {res_dist['linear_c']:.6f};")
    else: # Sigmoide
        print(f"double k_dist = {res_dist['sig_k']:.6f};")
        print(f"double mu_dist = {res_dist['sig_center']:.6f};")

    print("\n@Override")
    print("public double calculateReward(SensorModel sensors) {")
    print("    double speed = sensors.getSpeed();")
    print("    double dist = sensors.getTrackEdgeSensors()[9];")
    print("    double trackPos = sensors.getTrackPosition();")
    print("")
    print("    // 1. Penalización fuerte por salida")
    print("    if (Math.abs(trackPos) > 0.98) return -10.0;")
    print("")
    print("    // 2. Cálculo de Recompensa Normalizada")
    
    # Escribir la ecuación de velocidad
    if res_speed['type'] == 'LINEAL':
        print(f"    double r_speed = (m_speed * speed) + c_speed;")
    else:
        # Sigmoide formula: min + (max-min) / (1 + exp(-k*(x-mu)))
        scale = res_speed['max_out'] - res_speed['min_out']
        print(f"    double r_speed = {res_speed['min_out']} + ({scale} / (1.0 + Math.exp(-k_speed * (speed - mu_speed))));")

    # Escribir la ecuación de distancia
    if res_dist['type'] == 'LINEAL':
        print(f"    double r_dist = (m_dist * dist) + c_dist;")
    else:
        scale = res_dist['max_out'] - res_dist['min_out']
        print(f"    double r_dist = {res_dist['min_out']} + ({scale} / (1.0 + Math.exp(-k_dist * (dist - mu_dist))));")

    print("")
    print("    // 3. Mezcla de recompensas (Pesos sugeridos: 70% Speed, 30% Safety)")
    print("    // Como ambos r_speed y r_dist están normalizados por tus datos, se pueden sumar directamente.")
    print("    return (0.7 * r_speed) + (0.3 * r_dist);")
    print("}")

except Exception as e:
    print(f"Error: {e}")

Leyendo dataset...

>>> ANÁLISIS PARA: S_speed
    Mediana (Q2): 78.7778
    IQR (Q3-Q1):  54.4004
    Asimetría:    0.3058 (0=Normal, >1=Sesgada)
    RECOMENDACIÓN: La distribución es LINEAL.

>>> ANÁLISIS PARA: S_track_9
    Mediana (Q2): 42.5936
    IQR (Q3-Q1):  38.6577
    Asimetría:    1.7611 (0=Normal, >1=Sesgada)
    RECOMENDACIÓN: La distribución es SIGMOIDE.

 GENERADOR DE CÓDIGO JAVA (Copiar y Pegar en EnvAccel.java)

// Variables calculadas automáticamente para LINEAL y SIGMOIDE
// --- Lógica para Velocidad (LINEAL) ---
double m_speed = 0.018382;
double c_speed = -0.869194;
// Formula: reward = m * speed + c

// --- Lógica para Distancia (SIGMOIDE) ---
double k_dist = 0.103472;
double mu_dist = 42.593600;

@Override
public double calculateReward(SensorModel sensors) {
    double speed = sensors.getSpeed();
    double dist = sensors.getTrackEdgeSensors()[9];
    double trackPos = sensors.getTrackPosition();

    // 1. Penalización fuerte por salida
    if (Math.abs(trackPos)